In [1]:
import pandas as pd

# Read the compressed CSV file
# Load all four months' data and concatenate into a single dataframe
import glob

file_paths = [
    '../data1/flightlist_20200101_20200131.csv.gz',
    '../data1/flightlist_20200201_20200229.csv.gz',
    '../data1/flightlist_20200301_20200331.csv.gz',
    '../data1/flightlist_20200401_20200430.csv.gz'
]

all_dfs = [pd.read_csv(fp, compression='gzip') for fp in file_paths]
df = pd.concat(all_dfs, ignore_index=True)

# Display basic information about the dataframe
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

Shape: (8378688, 16)

Columns: ['callsign', 'number', 'icao24', 'registration', 'typecode', 'origin', 'destination', 'firstseen', 'lastseen', 'day', 'latitude_1', 'longitude_1', 'altitude_1', 'latitude_2', 'longitude_2', 'altitude_2']

First few rows:


,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,SXI1963,NaN,7c1ace,VH-FKW,F50,YPAD,WSSL,2019-12-31 00:07:07+00:00,2020-01-01 09:01:00+00:00,2020-01-01 00:00:00+00:00,-34.959753,138.474712,609.6,1.388535,103.849198,182.88
1,THY183,NaN,4ba9c1,TC-JNA,A332,NaN,LTBW,2019-12-31 00:25:26+00:00,2020-01-01 07:48:41+00:00,2020-01-01 00:00:00+00:00,42.134524,26.993280,7924.8,41.147849,28.709647,716.28
2,CES771,MU771,781858,B-30CW,A359,YSSY,EHAM,2019-12-31 01:29:25+00:00,2020-01-01 04:08:28+00:00,2020-01-01 00:00:00+00:00,-33.924545,151.170137,304.8,52.314357,4.765359,-160.02
3,AMX037,NaN,0d09e5,XA-ADC,B789,SAEZ,LEBL,2019-12-31 03:08:58+00:00,2020-01-01 10:05:29+00:00,2020-01-01 00:00:00+00:00,-34.825310,-58.518505,0.0,41.389572,2.349951,533.40
4,ACA43,NaN,c0173f,C-FIUW,B77W,CYYZ,CYYZ,2019-12-31 03:13:16+00:00,2020-01-01 10:25:51+00:00,2020-01-01 00:00:00+00:00,43.694653,-79.634288,0.0,43.675953,-79.610990,259.08


In [4]:
import urllib.request
import ssl
import certifi

url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
output_file = "../data1/airports.dat"

# Create SSL context with certifi certificates
context = ssl.create_default_context(cafile=certifi.where())

# Download with proper SSL context
with urllib.request.urlopen(url, context=context) as response:
    with open(output_file, 'wb') as out_file:
        out_file.write(response.read())

print(f"File downloaded successfully to {output_file}")

File downloaded successfully to ../data1/airports.dat


In [2]:
import pandas as pd

# Your column names
columns = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 
           'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 
           'Tz database time zone', 'Type', 'Source']

# Load the .dat file
airports_df = pd.read_csv('../data1/airports.dat', 
                          names=columns, 
                          header=None, 
                          na_values='\\N')

# Display info about the dataframe
print(f"Shape: {airports_df.shape}")
print(f"\nFirst few rows:")
airports_df.head()

Shape: (7698, 14)

First few rows:


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [3]:
print(df[df['origin'].isna()].shape)
print(df[df['destination'].isna()].shape)
print(df[df['origin'] == df['destination']].shape)


(2148532, 16)
(1552318, 16)
(575001, 16)


In [4]:
df= df[~df['origin'].isna()]
df= df[~df['destination'].isna()]
df.shape

(5069219, 16)

In [5]:
df = df[df['origin'] != df['destination']]
df.shape

(4494218, 16)

In [6]:
df[df['destination'] == "?"].shape

(0, 16)

In [7]:
# We'll perform two merges: one to add origin airport info, one for destination airport info.
# 'airports_in_usa' master is joined on ICAO (OpenFlights), matching to the flights df's 'origin' and 'destination' columns.
#df=df[df['origin'] != df['destination']]
# Merge for origin airport
flights_with_origin = df.merge(
    airports_df[['ICAO', 'Name', 'City', 'Country', 'IATA', 'Latitude', 'Longitude', 'Timezone']],
    left_on='origin',
    right_on='ICAO',
    how='left',
    suffixes=('', '_origin')
)

# Merge for destination airport
flights_with_airport_info = flights_with_origin.merge(
    airports_df[['ICAO', 'Name', 'City', 'Country', 'IATA', 'Latitude', 'Longitude', 'Timezone']],
    left_on='destination',
    right_on='ICAO',
    how='left',
    suffixes=('', '_destination')
)
# Find flights where at least one of origin or destination is in United States
#flights_with_airport_info_in_USA = flights_with_airport_info[
#    (flights_with_airport_info['Country'] == 'United States') |
#    (flights_with_airport_info['Country_destination'] == 'United States')
#]

# Display shape and preview
flights_with_airport_info.shape

(4494218, 32)

#2only international flights

In [ ]:
#internation_flights_usa = flights_with_airport_info_in_USA[flights_with_airport_info_in_USA['Country'] != flights_with_airport_info_in_USA['Country_destination']]
#internation_flights_usa.shape

(1870210, 32)

In [8]:
pd.to_datetime(flights_with_airport_info['day']).dt.month.value_counts()

day
2    1448833
1    1414183
3    1173180
4     458022
Name: count, dtype: int64

In [9]:
# Save flights_with_airport_info DataFrame to a TSV file
flights_with_airport_info.to_csv('flights_with_airport_info.tsv', sep='\t', index=False)

In [15]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Create a clean dataframe with only valid origin and destination city-country pairs
# Remove rows where city or country information is missing
graph_data = flights_with_airport_info[
    flights_with_airport_info['City'].notna() & 
    flights_with_airport_info['Country'].notna() & 
    flights_with_airport_info['City_destination'].notna() & 
    flights_with_airport_info['Country_destination'].notna()
].copy()

# Create node identifiers as (City, Country) tuples
graph_data['origin_node'] = list(zip(graph_data['City'], graph_data['Country']))
graph_data['destination_node'] = list(zip(graph_data['City_destination'], graph_data['Country_destination']))

# Count flights between each origin-destination pair
edge_counts = graph_data.groupby(['origin_node', 'destination_node']).size().reset_index(name='flight_count')

# Create a directed graph (use nx.Graph() for undirected)
G = nx.DiGraph()

# Add edges with weights (flight counts)
for _, row in edge_counts.iterrows():
    G.add_edge(row['origin_node'], row['destination_node'], weight=row['flight_count'])

# Print graph statistics
print(f"Number of nodes (cities): {G.number_of_nodes()}")
print(f"Number of edges (routes): {G.number_of_edges()}")
print(f"Total flights represented: {edge_counts['flight_count'].sum()}")

# Get top 10 busiest routes
top_routes = edge_counts.nlargest(10, 'flight_count')
print("\nTop 10 busiest routes:")
for _, route in top_routes.iterrows():
    origin = route['origin_node']
    dest = route['destination_node']
    count = route['flight_count']
    print(f"  {origin[0]}, {origin[1]} -> {dest[0]}, {dest[1]}: {count} flights")

# Get top 10 most connected cities (by degree)
degree_centrality = nx.degree_centrality(G)
top_cities = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 most connected cities:")
for city, centrality in top_cities:
    print(f"  {city[0]}, {city[1]}: {G.degree(city)} connections")

Number of nodes (cities): 2532
Number of edges (routes): 89017
Total flights represented: 3403460

Top 10 busiest routes:
  Sydney, Australia -> Melbourne, Australia: 6859 flights
  Melbourne, Australia -> Sydney, Australia: 6690 flights
  Delhi, India -> Mumbai, India: 5490 flights
  Mumbai, India -> Delhi, India: 5080 flights
  St. Petersburg, Russia -> Moscow, Russia: 4801 flights
  Toronto, Canada -> Montreal, Canada: 4522 flights
  Rio De Janeiro, Brazil -> Sao Paulo, Brazil: 4518 flights
  Montreal, Canada -> Toronto, Canada: 4512 flights
  Los Angeles, United States -> San Francisco, United States: 4446 flights
  Tokyo, Japan -> Fukuoka, Japan: 4323 flights

Top 10 most connected cities:
  London, United Kingdom: 939 connections
  Paris, France: 868 connections
  Teterboro, United States: 824 connections
  Atlanta, United States: 813 connections
  Chicago, United States: 804 connections
  Washington, United States: 761 connections
  Miami, United States: 748 connections
  Housto